# Web Server Log Analysis #

De forma general, un server log es un archivo de log generado por el servidor con una 
lista de las actividades que se ejecutan. En este caso tenemos un web server log el cuál 
mantiene un historial de las peticiones realizadas a la página. Este tipo de server logs 
tienen un formato standard (Common Log Format). Y es una práctica general, el 
analizar estos logs para sacar distintas conclusiones, localizar ataques, errores 
comunes, etc.

Primero debemos cargar el archivo como un archivo de texto normal y realizar las 
transformaciones pertinentes, a la hora de limpiar y estructurar nuestro dataset utilizaremos 
expresiones regulares para recoger los campos que necesitamos. 
Guardaremos nuestro nuevo DataFrame ya estructurado en formato parquet. Y de este 
leeremos para realizar nuestro análisis.

In [0]:
from pyspark.sql.session import SparkSession
import re

spark = (SparkSession
         .builder
         .appName("nasa")
         .getOrCreate())

In [0]:
from pyspark.sql.functions import *

In [0]:
base_df=spark.read.text("/FileStore/tables/access.log")
base_df.printSchema()


root
 |-- value: string (nullable = true)



In [0]:
host='^(\S+)\s'
ts_pattern='(\d{2}\/\w{3}\/\d{4}\:\d{2}:\d{2}:\d{2})'
method_uri_protocol_pattern='\"(\S+)\s(\S+)\s*(\S*)\"'
status_pattern='\s(\d{3})\s'
content_size_pattern='\s(\d+)$'

from pyspark.sql.functions import regexp_extract

logs_df=base_df.select(regexp_extract('value',host,1).alias('host'),
                        regexp_extract('value',ts_pattern,1).alias('timestamp'),
                        regexp_extract('value',method_uri_protocol_pattern,1).alias('method'),
                        regexp_extract('value',method_uri_protocol_pattern,2).alias('endpoint'),
                        regexp_extract('value',method_uri_protocol_pattern,3).alias('protocol'),
                        regexp_extract('value',status_pattern,1).cast('integer').alias('status'),
                        regexp_extract('value',content_size_pattern,1).cast('integer').alias('content_size'))

display(logs_df)

host,timestamp,method,endpoint,protocol,status,content_size
in24.inetnebr.com,01/Aug/1995:00:00:01,GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,HTTP/1.0,200,1839
uplherc.upl.com,01/Aug/1995:00:00:07,GET,/,HTTP/1.0,304,0
uplherc.upl.com,01/Aug/1995:00:00:08,GET,/images/ksclogo-medium.gif,HTTP/1.0,304,0
uplherc.upl.com,01/Aug/1995:00:00:08,GET,/images/MOSAIC-logosmall.gif,HTTP/1.0,304,0
uplherc.upl.com,01/Aug/1995:00:00:08,GET,/images/USA-logosmall.gif,HTTP/1.0,304,0
ix-esc-ca2-07.ix.netcom.com,01/Aug/1995:00:00:09,GET,/images/launch-logo.gif,HTTP/1.0,200,1713
uplherc.upl.com,01/Aug/1995:00:00:10,GET,/images/WORLD-logosmall.gif,HTTP/1.0,304,0
slppp6.intermind.net,01/Aug/1995:00:00:10,GET,/history/skylab/skylab.html,HTTP/1.0,200,1687
piweba4y.prodigy.com,01/Aug/1995:00:00:10,GET,/images/launchmedium.gif,HTTP/1.0,200,11853
slppp6.intermind.net,01/Aug/1995:00:00:11,GET,/history/skylab/skylab-small.gif,HTTP/1.0,200,9202


In [0]:
from pyspark.sql.functions import udf

month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(text):
    """ Convert Common Log time format into a Python datetime object
    Args:
        text (str): date and time in Apache time format [dd/mmm/yyyy:hh:mm:ss (+/-)zzzz]
    Returns:
        a string suitable for passing to CAST('timestamp')
    """
    # NOTE: We're ignoring the time zones here, might need to be handled depending on the problem you are solving
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(text[7:11]),
      month_map[text[3:6]],
      int(text[0:2]),
      int(text[12:14]),
      int(text[15:17]),
      int(text[18:20])
    )

In [0]:
udf_parse_time = udf(parse_clf_time)

logs_dft = logs_df.select('*', udf_parse_time(logs_df['timestamp']).cast('timestamp').alias('datetime')).drop('timestamp')
logs_dft.show(10, truncate=True)

+--------------------+------+--------------------+--------+------+------------+-------------------+
|                host|method|            endpoint|protocol|status|content_size|           datetime|
+--------------------+------+--------------------+--------+------+------------+-------------------+
|   in24.inetnebr.com|   GET|/shuttle/missions...|HTTP/1.0|   200|        1839|1995-08-01 00:00:01|
|     uplherc.upl.com|   GET|                   /|HTTP/1.0|   304|           0|1995-08-01 00:00:07|
|     uplherc.upl.com|   GET|/images/ksclogo-m...|HTTP/1.0|   304|           0|1995-08-01 00:00:08|
|     uplherc.upl.com|   GET|/images/MOSAIC-lo...|HTTP/1.0|   304|           0|1995-08-01 00:00:08|
|     uplherc.upl.com|   GET|/images/USA-logos...|HTTP/1.0|   304|           0|1995-08-01 00:00:08|
|ix-esc-ca2-07.ix....|   GET|/images/launch-lo...|HTTP/1.0|   200|        1713|1995-08-01 00:00:09|
|     uplherc.upl.com|   GET|/images/WORLD-log...|HTTP/1.0|   304|           0|1995-08-01 00:00:10|


In [0]:

logs_dft.write.parquet("/FileStore/tables/nasa_bien.parquet") 
nasa_df=spark.read.parquet("/FileStore/tables/nasa_bien.parquet")


In [0]:
display(nasa_df)

host,method,endpoint,protocol,status,content_size,datetime
van01028.direct.ca,GET,/images/USA-logosmall.gif,HTTP/1.0,200,234,1995-08-25T21:46:44.000+0000
van01028.direct.ca,GET,/images/WORLD-logosmall.gif,HTTP/1.0,200,669,1995-08-25T21:46:45.000+0000
205.233.69.3,GET,/ksc.html,HTTP/1.0,200,7089,1995-08-25T21:46:46.000+0000
205.233.69.3,GET,/images/NASA-logosmall.gif,HTTP/1.0,200,786,1995-08-25T21:46:48.000+0000
205.233.69.3,GET,/images/MOSAIC-logosmall.gif,HTTP/1.0,200,363,1995-08-25T21:46:48.000+0000
205.233.69.3,GET,/images/USA-logosmall.gif,HTTP/1.0,200,234,1995-08-25T21:46:48.000+0000
slip-26.cencar.udg.mx,GET,/images/kscmap.gif,HTTP/1.0,200,177415,1995-08-25T21:46:49.000+0000
205.233.69.3,GET,/images/WORLD-logosmall.gif,HTTP/1.0,200,669,1995-08-25T21:46:50.000+0000
up2-cs1p5.und.nodak.edu,GET,/ksc.html,HTTP/1.0,200,7089,1995-08-25T21:46:51.000+0000
up2-cs1p5.und.nodak.edu,GET,/images/ksclogo-medium.gif,HTTP/1.0,200,5866,1995-08-25T21:46:53.000+0000


¿Cuáles son los distintos protocolos web utilizados? Agrúpalos

>_Nota: Los protocolos WEB es la columna protocol_

In [0]:
nasa_df.groupBy("protocol").count().show()

+---------+-------+
| protocol|  count|
+---------+-------+
|         |   2765|
|HTTP/V1.0|    163|
| HTTP/1.0|1566969|
|        a|      1|
+---------+-------+



¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos 
para ver cuál es el más común.
>_Nota: Los códigos de estado son la columna status_

In [0]:
from pyspark.sql.functions import desc

cod_est=nasa_df.groupBy("status").count()
cod_est.orderBy(desc("count")).show()

+------+-------+
|status|  count|
+------+-------+
|   200|1398988|
|   304| 134146|
|   302|  26497|
|   404|  10056|
|   403|    171|
|   501|     27|
|   400|     10|
|   500|      3|
+------+-------+



¿Y los métodos de petición (verbos) más utilizados?

In [0]:
met_est=nasa_df.groupBy("method").count()
met_est.orderBy(desc("count")).show()

+---------------+-------+
|         method|  count|
+---------------+-------+
|            GET|1564929|
|           HEAD|   3965|
|               |    891|
|           POST|    111|
|�|t�9ð'À|u|      2|
+---------------+-------+



¿Qué recurso tuvo la mayor transferencia de bytes de la página web?

>_Nota: recurso en columna endpoint, transferencia de bytes en content_size_

In [0]:
rec_est=nasa_df.groupBy("endpoint").sum("content_size").orderBy(desc("sum(content_size)"))
display(rec_est)

endpoint,sum(content_size)
/shuttle/missions/sts-71/movies/sts-71-launch.mpg,1639380464
/shuttle/missions/sts-69/count69.gif,1005927794
/shuttle/missions/sts-69/movies/sts-69-rollback.mpg,512058235
/shuttle/technology/sts-newsref/stsref-toc.html,493211198
/shuttle/missions/sts-69/movies/ws-animation-deploy.mpg,464050354
/shuttle/missions/sts-53/movies/sts-53-launch.mpg,439216472
/shuttle/countdown/video/livevideo2.gif,407684744
/images/rss.gif,324585755
/images/ksclogo-medium.gif,320524106
/shuttle/missions/sts-71/movies/sts-71-mir-dock.mpg,309568460


Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es 
decir, el recurso con más registros en nuestro log

In [0]:
host_est=nasa_df.groupBy("endpoint").count().orderBy("count",ascending=False)
display(host_est)

endpoint,count
/images/NASA-logosmall.gif,97384
/images/KSC-logosmall.gif,75332
/images/MOSAIC-logosmall.gif,67441
/images/USA-logosmall.gif,67061
/images/WORLD-logosmall.gif,66437
/images/ksclogo-medium.gif,62771
/ksc.html,43683
/history/apollo/images/apollo-logo1.gif,37824
/images/launch-logo.gif,35135
/,30328


¿Qué días la web recibió más tráfico?

In [0]:
# Hago un df que contiene la fecha en formato yyyy/MM/dd
nasa_dfdate = (nasa_df
    .withColumn("date", date_format("datetime", "yyyy/MM/dd"))
    .select("*"))

# Ahora agrupamos por fecha y en cada una sumamos el content_size, ordenándolo de mayor a menor tamaño. Así, el día que más tráfico se recibió fue el 31 de Agosto.
nasa_data=nasa_dfdate.select("date","content_size").groupBy("date").sum("content_size").sort("sum(content_size)",ascending=False)
display(nasa_data)


date,sum(content_size)
1995/08/31,1426924103
1995/08/30,1288732734
1995/08/04,1109750886
1995/08/11,1102726342
1995/08/29,1089229521
1995/08/14,1083538174
1995/08/08,1063204146
1995/08/15,1054416847
1995/08/17,1045943479
1995/08/10,1034849598


¿Cuáles son los hosts son los más frecuentes?

In [0]:
# A continuación mostramos los 5 host más frecuentes.
nasa_df.groupBy("host").count().sort("count",ascending=False).show(5,truncate=False)

+--------------------+-----+
|host                |count|
+--------------------+-----+
|edams.ksc.nasa.gov  |6530 |
|piweba4y.prodigy.com|4846 |
|163.206.89.4        |4791 |
|piweba5y.prodigy.com|4607 |
|piweba3y.prodigy.com|4416 |
+--------------------+-----+
only showing top 5 rows



¿A qué horas se produce el mayor número de tráfico en la web?

In [0]:
# Añadimos una columna que extrae la hora de cada fecha de datetime
nasa_hours = (nasa_df
              .withColumn("hour", hour("datetime")))

# Agrupamos por horas y en cada una sumamos el content_size, ordenándolo de mayor a menor.
nasa_hours.select("hour","content_size").groupBy("hour").sum("content_size").orderBy("sum(content_size)",ascending=False).show(5,truncate=False)

+----+-----------------+
|hour|sum(content_size)|
+----+-----------------+
|13  |1800109838       |
|15  |1766223379       |
|12  |1762842005       |
|16  |1726466784       |
|14  |1637945671       |
+----+-----------------+
only showing top 5 rows



¿Cuál es el número de errores 404 que ha habido cada día?

In [0]:
nasa_errores=nasa_dfdate.select("status","date").where(nasa_dfdate.status == '404')
display(nasa_errores.groupBy("date").count().sort("count",ascending=False))

date,count
1995/08/30,571
1995/08/07,537
1995/08/31,526
1995/08/24,420
1995/08/29,420
1995/08/25,415
1995/08/28,410
1995/08/08,391
1995/08/06,373
1995/08/27,370
